In [1]:
import pandas as pd

from prophet import Prophet
from sklearn.model_selection import TimeSeriesSplit

ModuleNotFoundError: No module named 'prophet'

In [2]:
FLIGHT_SALES_PATH = '/home/chk/Documents/univ/maestria/dataton/Datathon/flight_sales.csv'

In [4]:
dtype_options = {
    'Unnamed: 0': 'int64',
    'Flight_ID': 'str',
    'ProductType': 'str',
    'ProductName': 'str',
    'Quantity': 'float64',
    'TotalSales': 'float64',
    'Aeronave': 'str',
    'DepartureStation': 'str',
    'ArrivalStation': 'str',
    'DestinationType': 'str',
    'Origin_Type': 'str',
    'Capacity': 'float64',
    'Passengers': 'float64',
    'Bookings': 'float64'
}

parse_dates = ['STD', 'STA']

flightSales = pd.read_csv(FLIGHT_SALES_PATH, dtype=dtype_options, parse_dates=parse_dates)
flightSales['STD'] = pd.to_datetime(flightSales['STD'])
flightSales['STA'] = pd.to_datetime(flightSales['STA'])
flightSales.head()

,Unnamed: 0,Flight_ID,ProductType,ProductName,Quantity,TotalSales,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
0,0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1.0,55.0,XA-VIQ,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186.0,131.0,100.0
1,1,a05290288259526edd3601160b10e1de,Botanas,Cheetos,2.0,98.0,XA-VIQ,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186.0,131.0,100.0
2,2,a05290288259526edd3601160b10e1de,Botanas,Ruffles Queso,6.0,294.0,XA-VIQ,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186.0,131.0,100.0
3,3,a05290288259526edd3601160b10e1de,Refrescos,Coca Sin Azucar,2.0,96.0,XA-VIQ,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186.0,131.0,100.0
4,4,a05290288259526edd3601160b10e1de,Licores,Jack And Coke,1.0,72.0,XA-VIQ,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186.0,131.0,100.0


In [14]:
flightSales['Duration'] = (flightSales['STA'] - flightSales['STD']).dt.total_seconds() / 3600
flightSales['TravelType'] = flightSales['Duration'].apply(lambda x: 'corto' if x < 2.5 else 'largo')
flightSales['PassengerRate'] = flightSales['Passengers'] / flightSales['Capacity'] * 100

In [16]:
salesGrouped = flightSales.groupby(['Origin_Type', 'Destination_Type', 'TravelType', 'STD']).agg({
    'PassengerRate': 'mean',
    'Duration': 'mean',
})
salesGrouped.reset_index(inplace=True)
salesGrouped.rename(columns={'STD': 'ds', 'PassengerRate': 'y'}, inplace=True)
salesGrouped

,Origin_Type,Destination_Type,TravelType,ds,y,Duration
0,Ciudad Fronteriza,Ciudad Principal,corto,2023-01-02 09:20:00,78.181818,2.0
1,Ciudad Fronteriza,Ciudad Principal,corto,2023-01-03 16:40:00,96.666667,2.0
2,Ciudad Fronteriza,Ciudad Principal,corto,2023-01-05 16:40:00,87.222222,2.0
3,Ciudad Fronteriza,Ciudad Principal,corto,2023-01-06 09:20:00,57.727273,2.0
4,Ciudad Fronteriza,Ciudad Principal,corto,2023-01-07 16:40:00,72.580645,2.0
...,...,...,...,...,...,...
227212,Playa,Playa,largo,2023-10-15 08:55:00,94.444444,3.5
227213,Playa,Playa,largo,2023-10-19 21:05:00,76.881720,3.5
227214,Playa,Playa,largo,2023-10-22 08:55:00,93.548387,3.5
227215,Playa,Playa,largo,2024-04-08 20:45:00,NaN,3.5


In [24]:
tscv = TimeSeriesSplit()

# for train_index, val_index in tscv.split(salesGrouped):
#     train_df = salesGrouped.iloc[train_index]
#     val_df = salesGrouped.iloc[val_index]

train_index, val_index = next(tscv.split(salesGrouped))
train_df = salesGrouped.iloc[train_index]
val_df = salesGrouped.iloc[val_index]

In [26]:
val_df

,Origin_Type,Destination_Type,TravelType,ds,y,Duration
37872,Ciudad Principal,Ciudad Fronteriza,largo,2024-09-22 21:40:00,NaN,2.666667
37873,Ciudad Principal,Ciudad Fronteriza,largo,2024-09-23 05:00:00,NaN,2.583333
37874,Ciudad Principal,Ciudad Fronteriza,largo,2024-09-23 06:20:00,NaN,2.666667
37875,Ciudad Principal,Ciudad Fronteriza,largo,2024-09-23 08:25:00,NaN,2.666667
37876,Ciudad Principal,Ciudad Fronteriza,largo,2024-09-23 15:00:00,NaN,2.583333
...,...,...,...,...,...,...
75736,Ciudad Principal,MX Amigos y Familia,corto,2023-03-17 18:55:00,71.250000,1.750000
75737,Ciudad Principal,MX Amigos y Familia,corto,2023-03-17 19:20:00,102.083333,1.166667
75738,Ciudad Principal,MX Amigos y Familia,corto,2023-03-17 19:45:00,103.333333,1.333333
75739,Ciudad Principal,MX Amigos y Familia,corto,2023-03-17 20:30:00,101.575024,1.416667
